## IoU及其优化变形

本笔记通过python实现IoU以及IoU的优化变形

### IoU

In [1]:
import tensorflow as tf

In [7]:
def IoU(box1, box2):
    '''
    iou loss
    :param box1: tensor [batch, w, h, num_anchor, 4], xywh 预测值，4表示每个检测框的中心点坐标(x, y)和预测框宽高(w, h)。
    :param box2: tensor [batch, w, h, num_anchor, 4], xywh 真实值
    :return: tensor [batch w, h, num_anchor, 1], xywh真实值
    '''
    box1_xy, box1_wh = box1[..., :2], box1[..., 2:4]
    box1_wh_half = box1_wh//2
    box1_min = box1_xy - box1_wh_half
    box1_max = box1_xy+box1_wh_half

    # 真实框处理
    box2_xy, box2_wh = box2[..., :2], box2[..., 2:4]
    box2_wh_half = box2_wh//2
    box2_min = box2_xy - box2_wh_half
    box2_max = box2_xy+box2_wh_half

    # 预测框面积
    box1_area = box1_wh[..., 0] * box1_wh[..., 1]
    box2_area = box2_wh[..., 0] * box2_wh[..., 1]

    # 找出交集区域的xy坐标
    intersect_min = tf.maximum(box1_min, box2_min)
    intersect_max = tf.minimum(box1_max, box2_max)
    intersect_wh = tf.maximum(intersect_max-intersect_min, 0)

    # 计算交集区域面积
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    # 计算并集区域面积
    union_area = box1_area+box2_area-intersect_area

    # 计算交并比，注意分母不能为0
    iou = intersect_area/(union_area+tf.keras.backend.epsilon())
    return iou

In [6]:
# Test

box1 = tf.fill([32, 16, 16, 3, 4], 50.0)
box2 = tf.fill([32, 16, 16, 3, 4], 40.0)
iou = IoU(box1, box2)
print(iou.shape)
# 查看某一张图的三个先验框
print(iou[0, 0, 0])


(32, 16, 16, 3)
tf.Tensor([0.42608696 0.42608696 0.42608696], shape=(3,), dtype=float32)


### GIoU

对于预测矩形框A 和 真实矩形框 B，计算能够同时包含A和B的最小的封闭矩形区域C。将封闭区域C 的面积减去 A和B的并集面积，再除以C的面积，得到一个比值。Giou就等于iou减去这个比值，公式如下：$$GIoU = IoU - \frac{C-(A \bigcup B)}{C}$$

GIoU损失的特点：

- 能够衡量两个边界框的巨鹿
- 不收目标对象大小的限制，具有很好的泛化能力
- 引入包含预测框A和真实框B的最小封闭区C，所以即使A和B不相交时，依然可以对检测框优化
- 不仅能反映边界框A和B是否有重叠区域，还能反映两个边界框是如何重叠的

In [9]:
def GIoU(box1, box2):
    '''
    iou loss
    :param box1: tensor [batch, w, h, num_anchor, 4], xywh 预测值，4表示每个检测框的中心点坐标(x, y)和预测框宽高(w, h)。
    :param box2: tensor [batch, w, h, num_anchor, 4], xywh 真实值
    :return: tensor [batch w, h, num_anchor, 1], xywh真实值
    '''
    box1_xy, box1_wh = box1[..., :2], box1[..., 2:4]
    box1_wh_half = box1_wh//2
    box1_min = box1_xy - box1_wh_half
    box1_max = box1_xy+box1_wh_half

    # 真实框处理
    box2_xy, box2_wh = box2[..., :2], box2[..., 2:4]
    box2_wh_half = box2_wh//2
    box2_min = box2_xy - box2_wh_half
    box2_max = box2_xy+box2_wh_half

    # 预测框面积
    box1_area = box1_wh[..., 0] * box1_wh[..., 1]
    box2_area = box2_wh[..., 0] * box2_wh[..., 1]

    # 找出交集区域的xy坐标
    intersect_min = tf.maximum(box1_min, box2_min)
    intersect_max = tf.minimum(box1_max, box2_max)
    intersect_wh = tf.maximum(intersect_max-intersect_min, 0)

    # 计算交集区域面积
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    # 计算并集区域面积
    union_area = box1_area+box2_area-intersect_area

    # 计算交并比，注意分母不能为0
    iou = intersect_area/(union_area+tf.keras.backend.epsilon())

    # 计算最小封闭矩形框
    enclose_min = tf.minimum(box1_min, box2_min)
    enclose_max = tf.maximum(box1_max, box2_max)
    enclose_wh = enclose_max - enclose_min
    # 面积 = ew*eh
    enclose_area = enclose_wh[..., 0] * enclose_wh[..., 1]
    giou = iou - (enclose_area-union_area)/(enclose_area+tf.keras.backend.epsilon())
    return iou, giou

In [12]:
_, giou = GIoU(box1, box2)
print(giou[0,0,0]) 

tf.Tensor([0.3765002 0.3765002 0.3765002], shape=(3,), dtype=float32)


### DIoU

GIoU的参数更新和优化会很缓慢，因此DIoU在这个基础上加入了中心点归一化，将预测框和真实框之间的距离、重叠率、尺度都考虑进去，能够直接最小化两个预测框之间的距离，使得目标边界框回归变得更加稳定，收敛速度更快。

DIoU的公式：$$DIoU = IoU - \frac{\rho^2(b, b^{gt})}{c^2} = IoU - \frac{d^2}{c^2}$$其中$b$表示预测框的中心点坐标， $b^{gt}$表示真实框的中心点坐标，$\rho$表示两个中心点之间的欧式距离，$c$表示两个目标边界框外接矩形的对角线长度。

In [18]:
def DIoU(box1, box2):
    '''
    iou loss
    :param box1: tensor [batch, w, h, num_anchor, 4], xywh 预测值，4表示每个检测框的中心点坐标(x, y)和预测框宽高(w, h)。
    :param box2: tensor [batch, w, h, num_anchor, 4], xywh 真实值
    :return: tensor [batch w, h, num_anchor, 1], xywh真实值
    '''
    box1_xy, box1_wh = box1[..., :2], box1[..., 2:4]
    box1_wh_half = box1_wh//2
    box1_min = box1_xy - box1_wh_half
    box1_max = box1_xy+box1_wh_half

    # 真实框处理
    box2_xy, box2_wh = box2[..., :2], box2[..., 2:4]
    box2_wh_half = box2_wh//2
    box2_min = box2_xy - box2_wh_half
    box2_max = box2_xy+box2_wh_half

    # 预测框面积
    box1_area = box1_wh[..., 0] * box1_wh[..., 1]
    box2_area = box2_wh[..., 0] * box2_wh[..., 1]

    # 找出交集区域的xy坐标
    intersect_min = tf.maximum(box1_min, box2_min)
    intersect_max = tf.minimum(box1_max, box2_max)
    intersect_wh = tf.maximum(intersect_max-intersect_min, 0)

    # 计算交集区域面积
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    # 计算并集区域面积
    union_area = box1_area+box2_area-intersect_area

    # 计算交并比，注意分母不能为0
    iou = intersect_area/(union_area+tf.keras.backend.epsilon())

    # 求出两个框的最小封闭矩形
    enclose_min = tf.minimum(box1_min, box2_min)  # 左上坐标
    enclose_max = tf.maximum(box1_max, box2_max)  # 右下坐标
    enclose_wh = enclose_max - enclose_min  # 封闭矩形的宽高
 
    # 计算对角线距离 w**2 + h**2
    enclose_distance = tf.square(enclose_wh[..., 0]) + tf.square(enclose_wh[..., 1])
 
    # ③ 计算两个框中心点之间的距离，计算方法同上
    center_distance = tf.reduce_sum(tf.square(box1_xy - box2_xy), axis=-1)
 
    # ④ 计算diou
    diou = iou - (center_distance / enclose_distance)
 
    # 返回每个检测框的iou和diou
    return iou, diou

In [19]:
_, diou = DIoU(box1, box2)
print(diou[0,0,0]) 

tf.Tensor([0.39302912 0.39302912 0.39302912], shape=(3,), dtype=float32)


### CIoU

作为一个优秀的回归定位损失应该考虑到三种几何参数：重叠面积、中心点距离、长宽比。CIOU损失关注了边界框长宽比的统一性，因此有更快的收敛速度和更好的性能。长宽比公式：$$\alpha = \frac{v}{1-IoU+v}, v=\frac{4}{\pi^2}(\arctan \frac{w^{gt}}{h^{gt}} - \arctan \frac{2}{h})^2$$ CIoU公式：$$IoU - (\frac{\rho^2(b, b^{gt})}{c^2}+ac)$$

In [24]:
import math
def CIoU(box1, box2):
    '''
    iou loss
    :param box1: tensor [batch, w, h, num_anchor, 4], xywh 预测值，4表示每个检测框的中心点坐标(x, y)和预测框宽高(w, h)。
    :param box2: tensor [batch, w, h, num_anchor, 4], xywh 真实值
    :return: tensor [batch w, h, num_anchor, 1], xywh真实值
    '''
    box1_xy, box1_wh = box1[..., :2], box1[..., 2:4]
    box1_wh_half = box1_wh//2
    box1_min = box1_xy - box1_wh_half
    box1_max = box1_xy+box1_wh_half

    # 真实框处理
    box2_xy, box2_wh = box2[..., :2], box2[..., 2:4]
    box2_wh_half = box2_wh//2
    box2_min = box2_xy - box2_wh_half
    box2_max = box2_xy+box2_wh_half

    # 预测框面积
    box1_area = box1_wh[..., 0] * box1_wh[..., 1]
    box2_area = box2_wh[..., 0] * box2_wh[..., 1]

    # 找出交集区域的xy坐标
    intersect_min = tf.maximum(box1_min, box2_min)
    intersect_max = tf.minimum(box1_max, box2_max)
    intersect_wh = tf.maximum(intersect_max-intersect_min, 0)

    # 计算交集区域面积
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    # 计算并集区域面积
    union_area = box1_area+box2_area-intersect_area

    # 计算交并比，注意分母不能为0
    iou = intersect_area/(union_area+tf.keras.backend.epsilon())

    # ② 求出包含两个框的最小封闭矩形
    enclose_min = tf.minimum(box1_min, box2_min)  # 左上坐标
    enclose_max = tf.maximum(box1_max, box2_max)  # 右下坐标
 
    # 计算对角线距离
    enclose_distance = tf.reduce_sum(tf.square(enclose_max - enclose_min), axis=-1)
 
    # 计算两个框中心点之间的距离，计算方法同上
    center_distance = tf.reduce_sum(tf.square(box1_xy - box2_xy), axis=-1)
    
    # ③ 考虑长宽比
    # tf.math.atan2()返回[-pi, pi]之间的角度
    v = 4 * tf.square(tf.math.atan2(box1_wh[..., 0], box1_wh[..., 1]) - tf.math.atan2(box2_wh[..., 0], box2_wh[..., 1])) / (math.pi * math.pi)
    alpha = v / (1.0 - iou + v)
 
    # 计算ciou
    ciou = iou - center_distance / enclose_distance - alpha * v
 
    return iou, ciou

In [25]:
_, ciou = CIoU(box1, box2)
print(ciou[0,0,0]) 

tf.Tensor([0.39302912 0.39302912 0.39302912], shape=(3,), dtype=float32)


### 参考

1. [【目标检测】(11) 预测框定位损失 iou、Giou、Diou、Ciou，附TensorFlow完整代码](https://blog.csdn.net/dgvv4/article/details/124039111)